In [1]:
import pandas as pd 
import numpy as np 
from keras.layers import Dense , SimpleRNN , GRU , LSTM , Embedding
from keras.models import sequential
from tensorflow.keras.utils import to_categorical

In [2]:
amazone = pd.read_csv(r"D:\Imarticus 2021\python\9th jan\amazonreviews.tsv" , sep = '\t')

In [3]:
amazone.shape
amazone = amazone.iloc[ : , [0,1]]
amazone.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [4]:
#amazone.rename(columns={"v1":"Label","v2":"Message"} , inplace = True)

In [5]:
amazone.label.value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [6]:
amazone.replace({"label":{"pos" :1,"neg" :0}} , inplace = True)

In [7]:
amazone_x = amazone.iloc[: ,1]
amazone_y = amazone.iloc[: ,0]

In [8]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train ,y_test = train_test_split(amazone_x , amazone_y , test_size=.2) 

In [9]:
x_train.shape

(8000,)

In [10]:
y_train.shape

(8000,)

In [11]:
y_test.shape

(2000,)

In [12]:
y_train

9100    1
7550    1
4842    0
582     1
9798    1
       ..
5252    0
5008    0
4078    1
5509    0
7126    1
Name: label, Length: 8000, dtype: int64

In [13]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [14]:
y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [15]:
max_num_words = 37500 ##how many words d we consider from the entire CORPUS
seq_len       = 210          ##How many thens(words to consider in each document/messahges)
embedding_size = 120   ##vector length of each word

In [16]:
from keras.preprocessing.text     import Tokenizer  #  is used for tokinizeation
from keras.preprocessing.sequence import pad_sequences 
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [17]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(amazone.review)

In [18]:
x_train = tokenizer.texts_to_sequences(x_train) # will convert the text to sequences of IDs
x_train = pad_sequences(x_train , maxlen = seq_len)

In [19]:
from keras.models import Sequential

In [20]:
x_test  =  tokenizer.texts_to_sequences(x_test)
x_test  =  pad_sequences(x_test , maxlen = seq_len)
model = Sequential()  # iniitialize the network
model.add(Embedding(input_dim = max_num_words,    
                   input_length = seq_len,
                    output_dim = embedding_size))

In [21]:
model.add(LSTM(5)) 
model.add(Dense(2 , activation  = 'softmax'))  # this 2 bcoz u have 2 level in target variable
adam = Adam(lr = .001)
model.compile(optimizer= 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

C:\Users\Rohan Bawaskar\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
model.fit(x_train , y_train , epochs = 5 , batch_size= 32 , validation_split= .2)

Epoch 1/5
200/200 [==============================] - 28s 132ms/step - loss: 0.5980 - accuracy: 0.6675 - val_loss: 0.4650 - val_accuracy: 0.7962
Epoch 2/5
200/200 [==============================] - 25s 125ms/step - loss: 0.2796 - accuracy: 0.8963 - val_loss: 0.3918 - val_accuracy: 0.8369
Epoch 3/5
200/200 [==============================] - 28s 138ms/step - loss: 0.1440 - accuracy: 0.9520 - val_loss: 0.4638 - val_accuracy: 0.8150
Epoch 4/5
200/200 [==============================] - 31s 157ms/step - loss: 0.0693 - accuracy: 0.9809 - val_loss: 0.5144 - val_accuracy: 0.8150
Epoch 5/5
200/200 [==============================] - 32s 162ms/step - loss: 0.0353 - accuracy: 0.9925 - val_loss: 0.5915 - val_accuracy: 0.8144


In [23]:
from sklearn.metrics import confusion_matrix

In [24]:
pred = model.predict(x_test)
pred

array([[5.1614203e-02, 9.4838578e-01],
       [1.2749124e-02, 9.8725080e-01],
       [9.0068024e-01, 9.9319726e-02],
       ...,
       [2.4664851e-02, 9.7533518e-01],
       [9.9921942e-01, 7.8058953e-04],
       [9.3390765e-03, 9.9066097e-01]], dtype=float32)

In [25]:
pred_classes = np.argmax(pred , axis = 1)
pred_classes

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [26]:
y_test_arg = np.argmax(y_test , axis = 1)
y_test_arg

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [27]:
confusion_matrix(y_test_arg ,pred_classes)

array([[831, 176],
       [152, 841]], dtype=int64)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_arg ,pred_classes)

0.836